In [2]:
from __future__ import division
from parser import PackToDocks
import codecs
import os
from wiki_ru_wordnet import WikiWordnet
import pymorphy2
from nltk.corpus import stopwords 
import nltk
import xlrd
from itertools import combinations

'''class_list_trump = PackToDocks('trump_labelled.txt', 'data_trump')

class_list_DNR = PackToDocks('lnr_dnr_labelled.txt', 'data_DNR')

stop_words = set(stopwords.words('russian')).remove(u'не')
'''

"class_list_trump = PackToDocks('trump_labelled.txt', 'data_trump')\n\nclass_list_DNR = PackToDocks('lnr_dnr_labelled.txt', 'data_DNR')\n\nstop_words = set(stopwords.words('russian')).remove(u'\xd0\xbd\xd0\xb5')\n"

# Это очень простая модель, предназначенная только лишь для сравнения

Суть модели в том, что из документа извлекаются сентиментные слова, имеющие определенные веса. Сумма весов определит класс, в который попадет документ. Всего выделим 3 класса: положительный, отрицательный и нейтральный. Веса прилагательных будет в 1.5 раз больше, чем у остальных слов.

В данной модели не будут использоваться деревья зависимостей слов, в связи с высоким временем построения. Так же для увеличения времени и качества работы алгоритма текст будет предварительно очищаться от стоп-слов и знаков препинания.

In [67]:
def ClearTable(table):
	for i in table:
		if i[0]==i[1]:
			table[i] == False
		else:
			table[i] == 0

def Add(dic, value):
	if value in dic:
		dic[value] +=1
	else:
		dic[value] = 1

def All(dic):
	for i in dic:
		if i[0] == i[1] and dic[i] == False:
			return False
	return True

def GetDict(cluster_list, class_list):
	s = set()
	d = {}
	for x in cluster_list:
		s.add(x)
	for kek in s:
		self_d = {}
		for i in s:
			self_d[i] = 0
		for i in range(len(cluster_list)):
			if cluster_list[i] == kek:
				Add(self_d, class_list[i])
		d[kek] = dict(self_d)
	return d
def Swap(cluster_list, class_list, max_key):
	for i in range(len(cluster_list)):
		if cluster_list[i] == max_key[0]:
			cluster_list[i] = max_key[1]
		elif cluster_list[i] == max_key[1]:
			cluster_list[i] = max_key[0]
def CPS(cluster_list, class_list):
	s = set()
	for x in cluster_list:
		s.add(x)
	table = {}
	for i in combinations(list(s), 2):
		table[i] = 0
	for i in s:
		table[(i, i)] = False
	while True:
		d = GetDict(cluster_list, class_list)
		for line in table:
			if line[0] != line[1]:
				table[line] = d[line[0]][line[1]] + d[line[1]][line[0]]
			else:
				table[line] = d[line[0]][line[1]]
		max_key = None
		for line in table:
			if table[line] > table[(line[0], line[0])] + table[(line[1], line[1])]:
				if max_key == None:
					max_key = line
				elif table[max_key] < table[line]:
					max_key = line
		if max_key == None:
			break
		else:
			Swap(cluster_list, class_list, max_key)

	counter = 0
	for i,x in enumerate(class_list):
		if x == cluster_list[i]:
			counter += 1
	return counter/len(class_list)

def Score(word, wordnet, workbook):
    worksheet = workbook.sheet_by_index(0)
    score = 0
    synsets = wordnet.get_synsets(word)
    if bool(synsets):
        synset = synsets[0]
        synlist = [i.lemma() for i in synset.get_words()]
    else:
        synlist = [word]
    counter = 0
    for token in synlist:
        flag = False
        for i in range(worksheet.nrows):
            if token == worksheet.cell(i, 0).value:
                counter +=1
                flag = True
                score += int(worksheet.cell(i, 1).value)
            elif flag:
                break
    if counter:
        score /= counter
    else:
        score = 0
    return score

def Clustering(data_path):
    file_list = [file for file in os.listdir(data_path) if os.path.isfile(data_path + '/' + file)]
    file_list.sort(key = lambda x: int(x[2:-4]))
    wordnet = WikiWordnet()
    workbook = xlrd.open_workbook('full word_rating_after_coding.xlsx')
    morph = pymorphy2.MorphAnalyzer()
    clusters = [0 for i in range(len(file_list))]
    for i, file in enumerate(file_list):
        with codecs.open(data_path + '/' + file, 'r', 'utf-8') as f:
            words = nltk.word_tokenize(f.read())
            words = [word.lower() for word in words if word.isalpha()]
            file_score = 0
            for j,word in enumerate(words):
                p = morph.parse(word)[0]
                score = Score(word = p.normal_form, workbook = workbook, wordnet = wordnet)
                if 'ADJF' in p.tag:
                    score *= 2
                if i != 0 and words[j-1] == u'не':
                    score *= -1
                file_score += score
            
            if file_score > 0.1:
                clusters[i] = 2
            elif file_score < -0.3:
                clusters[i] = 1
    return clusters


def PWS(class_list, cluster_list):
    PW = 0
    counter = 0
    for i, x in enumerate(class_list):
        for j, y in enumerate(class_list):
            if x == y:
                counter += 1
                PW += int(cluster_list[i] == cluster_list[j])

    return (PW/counter)


In [68]:
cluster_list_DNR = Clustering('data_DNR')
cluster_list_trump = Clustering('data_trump')

In [69]:
for i in range(len(class_list_DNR)):
    if class_list_DNR[i] == 9:
        class_list_DNR[i] = 0

print 'TRUMP CPS: ', CPS(class_list = class_list_trump, cluster_list = cluster_list_trump)
print 'DNR CPS: ', CPS(class_list = class_list_DNR, cluster_list = cluster_list_DNR)

print 'TRUMP PWS: ', PWS(class_list = class_list_trump, cluster_list = cluster_list_trump)
print 'DNR PWS: ', PWS(class_list = class_list_DNR, cluster_list = cluster_list_DNR)

print class_list_DNR
print '======================='
print cluster_list_DNR
print '---------------------------------------------------------'
print class_list_trump
print '======================='
print cluster_list_trump

TRUMP CPS:  0.466063348416
DNR CPS:  0.393939393939
TRUMP PWS:  0.497351897352
DNR PWS:  0.892867373245
[1, 0, 0, 0, 1, 0, 0, 2, 2, 1, 2, 2, 1, 1, 1, 0, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 2, 1, 2, 2, 1, 0, 1, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2, 1, 2, 2, 1, 2, 1, 0, 0, 1, 0, 1, 0, 2, 0, 2, 0, 1, 2, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 0, 2, 2, 2, 2, 1, 0, 1, 1, 0, 2, 2, 2, 2, 2, 2, 0, 0, 1, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---------------------------------------------------------
[0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 1, 0, 2, 1, 1, 0, 2, 1, 1, 0, 0, 1, 2, 1, 0, 1, 1, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 2, 1, 1, 0, 0, 0, 2, 2, 0, 1, 0, 2, 1, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 1, 1, 2, 2, 1, 0, 2

In [6]:
morph = pymorphy2.MorphAnalyzer()
p = morph.parse(u'стали')[0]
print p.normal_form

стать
